In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import json

# Custom Dataset
class ClaimClassificationDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_len=512):
        self.data = self.load_data(data_path)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_map = {'SUPPORTS': 0, 'REFUTES': 1, 'NOT_ENOUGH_INFO': 2, 'DISPUTED': 3}

    def load_data(self, data_path):
        with open(data_path, "r") as f:
            data = json.load(f)
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        claim = "CLAIM:" + item['claim_text']
        evidence = " [SEP] EVIDENCE:".join(item['evidences'])
        inputs = self.tokenizer(claim + evidence,
                                truncation=True, padding="max_length",  # Use dynamic padding
                                max_length=self.max_len, return_tensors='pt')
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs['labels'] = torch.tensor(self.label_map[item['claim_label']])
        return inputs

def create_dataloaders(train_path, dev_path, tokenizer, batch_size=16, max_len=512):
    
    train_dataset = ClaimClassificationDataset(train_path, tokenizer, max_len)
    dev_dataset = ClaimClassificationDataset(dev_path, tokenizer, max_len)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, dev_loader

In [ ]:
import torch
import torch.nn as nn
from torch.amp import GradScaler, autocast
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

# GradScaler
scaler = GradScaler()  

   
def train_one_epoch(model, train_loader, optimizer, scheduler, autocast_flag=False):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)

        optimizer.zero_grad()

        # Forward pass
        if autocast_flag:
            with autocast(device_type='cuda'):  # Updated autocast
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
        else:
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

         # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    return avg_loss

def evaluate(model, dev_loader):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dev_loader:
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            # Compute accuracy
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)
            correct_predictions += (preds == labels).sum().item()
            total_samples += labels.size(0)

            # Collect all predictions and labels for metrics calculation
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate average loss and accuracy
    avg_loss = total_loss / len(dev_loader)
    accuracy = correct_predictions / total_samples

    # Calculate precision, recall, and F1 score
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:\n", cm)

    # Return all metrics
    metrics = {
        "avg_loss": avg_loss,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }

    return metrics

def train_model(model, train_loader, dev_loader, epochs, optimizer, scheduler, best_model_path, autocast_flag):
    best_accuracy = 0

    print("START TRAINING: ")
    for epoch in range(epochs):
        print(f"----Epoch {epoch + 1}/{epochs}----")

        # Training
        train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, autocast_flag)
        print(f"Training Loss: {train_loss:.4f}")

        # Evaluation
        val_metrics = evaluate(model, dev_loader)
        print(f"Validation Loss:", val_metrics)

        # Save the best model
        if val_metrics["accuracy"] > best_accuracy:
            best_accuracy = val_metrics["accuracy"]
            best_model = model
            

    best_model.to('cpu')
    torch.save(best_model.state_dict(), best_model_path)



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
import time 

# Define hyperparameters
BATCH_SIZE = 4
EPOCHS = 10
MAX_LEN = 512
LEARNING_RATE = 5e-5

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAIN_PATH = "/kaggle/input/claim-evidence-pair/claim-evidence-set/claim-evidence-train_set.json"
DEV_PATH = "/kaggle/input/claim-evidence-pair/claim-evidence-set/claim-evidence-dev_set.json"

# Load DeBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=4)
model = model.to(DEVICE)

# Create data loaders
train_loader, dev_loader = create_dataloaders(TRAIN_PATH, DEV_PATH, tokenizer, batch_size=BATCH_SIZE, max_len=MAX_LEN)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_loader) * EPOCHS
num_warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=total_steps)

best_model_path = "/kaggle/working/deBERTa_v3_best_model.pt"

if __name__ == "__main__":
    start_time = time.time()
    train_model(model, train_loader, dev_loader, EPOCHS, optimizer, scheduler, best_model_path, autocast_flag=False)
    end_time = time.time()
    print("Model training time:", end_time-start_time)